# Data loading

In [146]:
def load_data(data_path):
    data = pd.read_csv(data_path)  
    return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
    print(len(pallet))
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
test = trainentest.iloc[273988:, :]



1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
4925535


# Python packages




In [4]:
import math, datetime, time, random
import numpy as np
import pandas as pd
from datetime import timedelta
import re
import math, datetime, time, random
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


# Door open




In [5]:
inbound = pd.read_csv('inbound_loads.csv')
ft = pd.read_csv('featuretrain.csv')

### Uitleg

1. Get all the minutes for the interval in between start and finish


In [106]:
def interval_min(start_date, end_date):
    dates_range = []
    while start_date <= end_date:
        dates_range.append(start_date) # or you can append str(start_date) if want the dates to be as strings
        start_date += timedelta(minutes=1)
    return dates_range

In [107]:
door = inbound.filter(['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], axis=1)
door.head(5)

,load_start_datetime,load_finish_datetime,dock_door_number
0,2019-02-11 04:42:00,2019-02-11 04:51:00,59.0
1,2019-01-16 10:11:00,2019-01-16 11:27:00,60.0
2,2019-07-30 09:59:00,2019-07-30 10:18:00,61.0
3,2019-01-10 08:51:00,2019-01-10 08:58:00,58.0
4,2019-01-10 07:26:00,2019-01-10 08:08:00,56.0


2. Get all the minutes intervals and put them in a new dataframe

In [123]:
door['load_finish_datetime'] = pd.to_datetime(door['load_finish_datetime'])
door['load_start_datetime'] = pd.to_datetime(door['load_start_datetime'])

door['load_minutes'] = np.nan

for index, row in door.iterrows():
    door['load_minutes'].iloc[index] = interval_min(door['load_start_datetime'].iloc[index], door['load_finish_datetime'].iloc[index])

C:\Users\suzan\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [177]:
door = door.drop_duplicates(subset=['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], keep="first")
door

,load_start_datetime,load_finish_datetime,dock_door_number,load_minutes
0,2019-02-11 04:42:00,2019-02-11 04:51:00,59.0,"[2019-02-11 04:42:00, 2019-02-11 04:43:00, 201..."
1,2019-01-16 10:11:00,2019-01-16 11:27:00,60.0,"[2019-01-16 10:11:00, 2019-01-16 10:12:00, 201..."
2,2019-07-30 09:59:00,2019-07-30 10:18:00,61.0,"[2019-07-30 09:59:00, 2019-07-30 10:00:00, 201..."
3,2019-01-10 08:51:00,2019-01-10 08:58:00,58.0,"[2019-01-10 08:51:00, 2019-01-10 08:52:00, 201..."
4,2019-01-10 07:26:00,2019-01-10 08:08:00,56.0,"[2019-01-10 07:26:00, 2019-01-10 07:27:00, 201..."
...,...,...,...,...
56135,2022-01-02 21:07:00,2022-01-02 21:20:00,54.0,"[2022-01-02 21:07:00, 2022-01-02 21:08:00, 202..."
56137,2022-01-03 00:38:00,2022-01-03 00:45:00,68.0,"[2022-01-03 00:38:00, 2022-01-03 00:39:00, 202..."
56138,2022-01-03 06:45:00,2022-01-03 06:57:00,42.0,"[2022-01-03 06:45:00, 2022-01-03 06:46:00, 202..."
56144,2022-01-03 16:12:00,2022-01-03 16:26:00,56.0,"[2022-01-03 16:12:00, 2022-01-03 16:13:00, 202..."


3. Make a flat list from the 'load_minutes' column 

In [178]:
def flatten(l):
    return [item for sublist in l for item in sublist]

list_min = door['load_minutes'].tolist()
list_min = flatten(list_min)

In [108]:
a = len(list_min)
print(a)

2176980


4. Create new dataframe with each time as an individual row (instead of list in each row) 

In [179]:
minutes = pd.DataFrame(list_min)

minutes.columns =['col1']
minutes

,col1
0,2019-02-11 04:42:00
1,2019-02-11 04:43:00
2,2019-02-11 04:44:00
3,2019-02-11 04:45:00
4,2019-02-11 04:46:00
...,...
1559988,2022-01-03 16:22:00
1559989,2022-01-03 16:23:00
1559990,2022-01-03 16:24:00
1559991,2022-01-03 16:25:00


5. Create a new dataframe with datetime and count

In [180]:
test = minutes.value_counts().rename_axis('datetime_local').to_frame('counts')
test.sort_values(by='datetime_local', inplace = True) 
test.head(100)
#test = pd.to_datetime(test.index)

,counts
datetime_local,
2018-12-30 06:33:00,1
2018-12-30 06:34:00,1
2018-12-30 06:35:00,1
2018-12-30 06:36:00,1
2018-12-30 06:37:00,1
...,...
2018-12-30 08:08:00,1
2018-12-30 08:09:00,1
2018-12-30 08:10:00,1


6. Now merge this dataframe with feature dataframe, on the datetime

In [145]:
train = trainentest.iloc[:273988,:]
test = trainentest.iloc[273988:, :]
def addtimecol(df, colname): ####input df and colname 
    df[colname] = pd.to_datetime(df[colname])         
    df['year'] = df[colname].dt.year
    df['month'] = df[colname].dt.month
    df['weekday'] = df[colname].dt.weekday
    df['day'] = df[colname].dt.day
    df['hour'] = df[colname].dt.hour
    df['minute'] = df[colname].dt.minute        
    return df

#Create new dummy dfs
base_df = train.copy()
base_weather = weather.copy()

#Remove unnecessary columns
base_df = base_df.drop('Unnamed: 0', axis=1)
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])
base_df['datetime_local'] = pd.to_datetime(base_df['datetime_local'])
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})
#base_weather['datetime_local'] = [datetime.datetime.strptime(x, 'yyyy/MM/dd HH:mm:SS') for x in base_weather['localstrptime']]

#Add time columns.
addtimecol(base_df, 'datetime_local')

#Set index to datetime
base_df.set_index('datetime_local', inplace=True)
base_weather.set_index('datetime_local', inplace=True)

#Concatenate the weather DataFrame to the base DataFrame
base_df = pd.concat([base_df, base_weather], axis=1)

# Drop all NaN values\n"
base_df.dropna(subset=['demand_kW', 'Temperature'])

NameError: name 'trainentest' is not defined

In [120]:
base_df = pd.concat([ft, test], axis=1)
#base_df["counts"] = base_df["counts"].fillna(0)

base_df
base_df.to_csv("feature_train_door_open")

In [153]:
type(test.index)
type(base_df.index)

pandas.core.indexes.base.Index

In [95]:
ft

,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC
datetime_local,,,,,,,,,,
2018-12-31 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:00:00-06:00,50.37,53.6,2019-01-01 00:00:00
2018-12-31 18:05:00,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:05:00-06:00,50.37,53.6,2019-01-01 00:05:00
2018-12-31 18:10:00,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:10:00-06:00,50.37,53.6,2019-01-01 00:10:00
2018-12-31 18:15:00,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:15:00-06:00,50.37,53.6,2019-01-01 00:15:00
2018-12-31 18:20:00,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:20:00-06:00,50.37,53.6,2019-01-01 00:20:00
...,...,...,...,...,...,...,...,...,...,...
2021-12-31 17:35:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31 17:35:00-06:00,82.89,68.0,2021-12-31 23:35:00
2021-12-31 17:40:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31 17:40:00-06:00,82.89,68.0,2021-12-31 23:40:00
2021-12-31 17:45:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31 17:45:00-06:00,82.89,68.0,2021-12-31 23:45:00


In [97]:
test

,counts
datetime_local,
2018-12-30 06:33:00,1
2018-12-30 06:34:00,1
2018-12-30 06:35:00,1
2018-12-30 06:36:00,1
2018-12-30 06:37:00,1
...,...
2022-01-03 17:19:00,1
2022-01-03 17:20:00,1
2022-01-03 17:21:00,1


In [118]:
base_df.iloc[400]

year                                       NaN
month                                      NaN
weekday                                    NaN
day                                        NaN
hour                                       NaN
minute                                     NaN
datetime             2019-01-02 00:40:00-06:00
Relative Humidity                        80.19
Temperature                               32.0
datetime_UTC               2019-01-02 06:40:00
counts                                     NaN
Name: 2019-01-02 00:40:00, dtype: object